# Infer CYTOSOL - part 3

--------------

## OBJECTIVE:  Infer sub-cellular component #3: CYTOSOL  in order to understand interactome 

To measure shape, position, and size of the cytosol of the cell body -- the soma minus the nucleus.    

Dependencies:
The CYTOSOL inference rely on the NUCLEI AND SOMA inference.  Therefore all of the sub-cellular objects rely on this segmentation.





# IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage import morphology
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.utils.file_io import read_input_image, list_image_files, export_ome_tiff
from infer_subc.utils.img import *

%load_ext autoreload
%autoreload 2

from infer_subc.organelles.cytosol import infer_CYTOSOL

# IMAGE PROCESSING Objective 3:  infer CYTOSOL
  [OUTLINE: Objective #2](#summary-of-objectives)
## summary of steps

INPUT
- labeled NUCLEI (objective #1)
- labeled SOMA (objective #2)

PRE-PROCESSING
- erode Nuclei (shrink)

CORE PROCESSING
  - XOR(NUCLEI, SOMA)

- POST-PROCESSING
  - N/A

OUTPUT
- mask of CYTOSOL
- CYTOSOL object (intensity)



# INPUT

In [3]:
###################
# INPUT
###################
##  set up files
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

chan_name = 'nuclei'
out_path = data_root_path / "inferred_objects" 
object_name = 'NU_object'

NU_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff"  )
NU_object = NU_bioim.image
NU_labels = label(NU_object)


object_name = 'SO_object3'
label_name = 'SO_label3'

SO_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff" )
SO_object = SO_bioim.image

SO_bioim = read_input_image( out_path/ f"{label_name}.ome.tiff" )
SO_labels = SO_bioim.image
scale = SO_bioim.meta['scale']
'/Users/ahenrie/Projects/Imaging/mcz_subcell/data/inferred_objects/SO_object3.ome.tiff'

/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 

'/Users/ahenrie/Projects/Imaging/mcz_subcell/data/inferred_objects/SO_object3.ome.tiff'

## PRE-PROCESSING

In [15]:
###################
# PRE_PROCESSING
###################

#NU_eroded1 = morphology.binary_erosion(NU_object,  footprint=morphology.ball(3) )
NU_eroded = morphology.binary_erosion(NU_object)  



## CORE PROCESSING

In [ ]:

###################
# CORE_PROCESSING
###################

# logical  SO - NU_eroded
# CY_labels = np.zeros_like(SO_labels)
CY_object = np.logical_and(SO_object,~NU_eroded)
# CY_labels[CY_object] = 1

In [17]:


chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 

# test: does this export work?
object_name = 'CY_object'
CY_object_filen = export_ome_tiff(CY_object, meta_dict_t, object_name, str(out_path)+"/", curr_chan=0)



viewer = napari.view_image( #viewer.add_image(
    CY_object,
    scale=scale,
    opacity=0.3,
)    


['CY_object']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


# DEFINE `infer_CYTOSOL` function

In [23]:
# copy this to base.py for easy import

def infer_CYTOSOL(SO_object, NU_object, erode_NU = True):
    """
    Procedure to infer NUCLEI from linearly unmixed input.

    Parameters:
    ------------
    SO_object: np.ndarray
        a 3d image containing the NUCLEI signal

    NU_object: np.ndarray
        a 3d image containing the NUCLEI signal

    erode_NU: bool
        should we erode?

    Returns:
    -------------
    CY_object: np.ndarray (bool)
      
    """

    #NU_eroded1 = morphology.binary_erosion(NU_object,  footprint=morphology.ball(3) )
    if erode_NU:
        CY_object = np.logical_and(SO_object,~morphology.binary_erosion(NU_object) )
    else:
        CY_object = np.logical_and(SO_object,~NU_object)
    return CY_object


In [24]:
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

out_path = data_root_path / "inferred_objects" 
object_name = 'NU_object'

NU_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff"  )
NU_object = NU_bioim.image
NU_labels = label(NU_object)

object_name = 'SO_object3'
label_name = 'SO_label3'
SO_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff" )
SO_object = SO_bioim.image

SO_bioim = read_input_image( out_path/ f"{label_name}.ome.tiff" )
SO_labels = SO_bioim.image

scale = meta_dict_t['scale']

CYTO_object =  infer_CYTOSOL(SO_object, NU_object) 



In [ ]:

viewer = napari.view_image( #viewer.add_image(
    CYTO_object,
    scale=scale,
    opacity=0.3,
)    

In [ ]:


chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 

# test: does this export work?
object_name = 'CY_object'
CY_object_filen = export_ome_tiff(CY_object, meta_dict_t, object_name, str(out_path)+"/", curr_chan=0)

